In [2]:
%matplotlib inline
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split

import keras
import matplotlib.pyplot as plt
from rasterio.plot import show
import xarray as xr

import gdal
import rasterio
import glob
import os
import numpy as np
import pandas as pd
import pandas_profiling as pp

In [2]:
data = pd.read_csv('./satellite_dataset.csv')
data.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno
0,203.0,131.0,36.0,21.0,61.0,46.0,1,0,0,0
1,207.0,137.0,39.0,27.0,71.0,59.0,1,0,0,0
2,393.0,325.0,217.0,200.0,254.0,176.0,1,0,0,0
3,269.0,201.0,58.0,11.0,51.0,43.0,1,0,0,0
4,194.0,218.0,112.0,103.0,153.0,130.0,1,0,0,0


In [3]:
# pp.ProfileReport(data)

In [4]:
dataset_X = data.drop(columns=['wofs','bosque','wofs_bosque','ninguno'], axis=1)
dataset_X.head()

,blue,green,red,nir,swir1,swir2
0,203.0,131.0,36.0,21.0,61.0,46.0
1,207.0,137.0,39.0,27.0,71.0,59.0
2,393.0,325.0,217.0,200.0,254.0,176.0
3,269.0,201.0,58.0,11.0,51.0,43.0
4,194.0,218.0,112.0,103.0,153.0,130.0


In [5]:
dataset_Y = data[['wofs','bosque','wofs_bosque','ninguno']]
dataset_Y.head()

,wofs,bosque,wofs_bosque,ninguno
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_Y, test_size=0.2)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [7]:
# DEFINICIÓN DEL MODELO
# Hay dos maneras de construir modelos en Keras: Secuencial y Funcional
# El modelo secuencial permite construir modelos capa por capa.
# El modelo funcional permite construir modelos mas complicados.

# La capa Flaten transforma los datos de un arreglo bidimensional de 28x28 a un arreglo 
# unidimensional de 784 posiciones(28x28) esto solo formatea el conjutno de datos

# La capa Dense significa que cada neurona en una capa esta conectada a todas las neuronas 
# localizadas en la capa anterior y con todas en la siguiente capa.

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(128,input_dim=6, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

# CONFIGURACIÓN DE PARÁMETROS
# La compilación del modelo comprende la configuración de parámetros 
# usados durante el entrenamiento: algori tmo de optimización, medida 
# de exactitud, etc.

# optimizer: Adam es un algoitmo de optimización. Además es un método de tasa de aprendizaje adaptativo, 
# loss: calcula la diferencia entre la salida y la variable objetivo. Mide la precisión del modelo durante 
# el entrenamiento y queremos minimizar esta función.
# metrics: Son las métricas que se desan calcular durante el entrenamiento. mide la fracción de imágenes que 
# están clasificadas correctamente.

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen de parámetros del modelo
model.summary()

# ENTRENAMIENTO
model.fit(X_train, y_train, epochs=15)

# EVALUACIÓN
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               896       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total params: 1,412
Trainable params: 1,412
Non-trainable params: 0
_________________________________________________________________

Epoch 1/15
800000/800000 [==============================] - 21s 27us/step - loss: 0.2807 - accuracy: 0.9739
Epoch 2/15
800000/800000 [==============================] - 21s 27us/step - loss: 0.0514 - accuracy: 0.9810
Epoch 3/15
800000/800000 [==============================] - 21s 26us/step - loss: 0.0513 - accuracy: 0.9814
Epoch 4/15
800000/800000 [==============================] - 21s 26us/step - loss: 0.0490 - accuracy: 0.9822
Epoch 5/15
800000/800000 [==============================] - 21s 26us/step - loss:

In [8]:
model.save('my_model.h5')